In [33]:
%run ItemModel.ipynb
%run APIService.ipynb

In [34]:
import inspect, re
import pandas as pd
import csv

# Class DataCrawler

In [44]:
class DataCrawler(apiServiceDelegate):
    newest = 0
    keyword = "thoi trang nam"
    categoryids = ""
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    noneCategory = True
    APIService = APIService(newest, keyword, categoryids)
    
    i = 1
    def __init__(self, keyword, newest = 0, categoryids = ''):
        if categoryids == '':
            self.noneCategory = True
        else:
            self.noneCategory = False
        self.newest = newest
        self.keyword = keyword
        self.categoryids = categoryids
        self.APIService = APIService(self.newest, self.keyword, self.categoryids)
        self.APIService.delegate = self
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def crawlData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                check = self.APIService.get()
                if check == False: break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.ctime = item['ctime'] #time since 1970
            itemModel.time_now = int(time.time())
            itemModel.sell_time = itemModel.time_now - itemModel.ctime
            itemModel.historical_sold = item['historical_sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            if len(item['tier_variations']) > 0:
                itemModel.options = 1
                for tier_variation in item['tier_variations']:
                    itemModel.options *= len(tier_variation['options'])
            else:
                itemModel.options = 0
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.five_star = itemModel.rating_count[5]
            itemModel.four_star = itemModel.rating_count[4]
            itemModel.three_star = itemModel.rating_count[3]
            itemModel.two_star = itemModel.rating_count[2]
            itemModel.one_star = itemModel.rating_count[1]
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'category_id\tname\tshop_location\titem_public_time\titem_be_got_time\tsell_time\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\tfive_star\tfour_star\tthree_star\ttwo_star\tone_star\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discount\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'historical_sold\n')
    
        # Notice that categoryids is not required. Feel free to get data but uncomment below if you did not pass categoryids argument.
#       
        if self.noneCategory == True:
            self.categoryids = '0'
        
        for item in self.all_items:
                file.write(f'{self.categoryids}\t{item.name}\t{item.shop_location}\t{item.ctime}\t{item.time_now}\t{item.sell_time}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.five_star}\t'
                + f'{item.four_star}\t{item.three_star}\t{item.two_star}\t{item.one_star}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Crawling Data

In [42]:
crawler = DataCrawler(newest = 0, keyword = 'thoi trang nam', categoryids = '15139')

In [43]:
crawler.crawlData()


17
get! 1
get All Data!


In [30]:
# crawler.parseData()

In [32]:
# Check last request
print(json.dumps(crawler.APIService.json_pydata, indent = 3, sort_keys=True))

{
   "adjust": null,
   "algorithm": null,
   "disclaimer_infos": null,
   "error": null,
   "hint_keywords": null,
   "items": null,
   "json_data": null,
   "need_next_search": null,
   "nomore": null,
   "price_adjust": null,
   "query_rewrite": null,
   "reserved_keyword": null,
   "show_disclaimer": null,
   "suggestion_algorithm": null,
   "total_ads_count": null,
   "total_count": null,
   "version": "453b4ce4955c7c8068c54bc30a274f21"
}


In [31]:
# Check feature value
crawler.all_items[0].ctime

IndexError: list index out of range

In [8]:
# Test ParseData
print(crawler.all_items[0].name)
print(crawler.all_items[50].name)
print(crawler.all_items[1].name)
print(crawler.all_items[51].name)

Áo Hoodies Nam Dài Tay Có Nón Thời Trang Cỡ Lớn M-5Xl
Áo thu đông nam dài tay chất nỉ Thái phối logo ngực hàng may đep dày mịn mã SW21
Áo hoodie tay dài dáng rộng họa tiết độc đáo thời trang nam cá tính
Áo Sweater Áo DAS Dập Nổi Nỉ Trơn Cổ Tròn Unisex Nam Nữ


In [9]:
crawler.saveFile("ShoppeItem_thoiTrangNam_trangSucNam.tsv")

In [10]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_trangSucNam.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df



,category_id,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,...,price_before_discount,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,historical_sold
0,8945,Áo Hoodies Nam Dài Tay Có Nón Thời Trang Cỡ Lớn M-5Xl,Nước ngoài,1606462075,1610611281,4149206,True,29%,False,21,...,22000000000,None,15,1037,4,False,False,7,0,12
1,8945,Áo hoodie tay dài dáng rộng họa tiết độc đáo thời trang nam cá tính,Nước ngoài,1600849604,1610611281,9761677,True,38%,False,30,...,29800000000,None,119,14474,51,False,False,9,0,158
2,8945,Áo Hoodie Dáng Rộng Thời Trang Cao Cấp Cho Nam,Nước ngoài,1606465252,1610611281,4146029,True,50%,False,20,...,25583300000,None,10,674,3,False,False,9,0,11
3,8945,Áo Hoodie Thời Trang Size M-5XL Dành Cho Nam Vừa Với Cả Người Nặng 100kg,Nước ngoài,1606143501,1610611281,4467780,True,30%,False,42,...,21000000000,None,255,24494,51,False,False,8,0,169
4,8945,Áo thu đông nam❤️ Freeship 50K❤️ Áo nam dài tay nỉ poly Thái dày mịn may kỹ Das Stich SW381,Hà Nội,1606702026,1610611281,3909255,True,23%,False,10,...,12900000000,None,11,1841,7,False,False,8,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,8945,Áo Hoodies Có Nón Dài Tay Thời Trang Nam 100% C,Nước n,2702108773,1610611281,-1091497492,True,38%,False,30,...,56197018901,None,0,2,0,False,False,9,0,0
1146,8945,STANLEY Áo Hoodies Có Nón Dài Tay Thời Tran,Nước n,2049167618,1610611281,-438556337,True,38%,False,30,...,80804992092,None,0,0,0,True,False,9,0,0
1147,8945,Áo Hoodies Có Nón Dài Tay Thời Tran,Nước n,1611512489,1610611281,-901208,False,45%,False,28,...,33363317367,None,0,0,0,True,True,9,0,0
1148,8945,Áo Hoodies Có Nón Dài Tay Thời Tran,Nước n,2161396096,1610611281,-550784815,False,41%,False,30,...,85841687626,None,0,0,0,True,True,9,0,0


In [87]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [88]:
del items_df